# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
import numpy as np
import copy

from kesacco import clustpipe

# Parameters

### Define the simulation

In [2]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example')

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [3]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.017284
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.2, 'r_c1':57*u.kpc, 'n_01':0.046*u.cm**-3,
                                   'beta2':0.71, 'r_c2':278*u.kpc, 'n_02':0.0036*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.X_cre1_E = {'X':0.0, 'R_norm':cpipe.cluster.R500} # No primary CRe
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isobaric_scal_param(scal=0.5)     # Assumes CR follow n_CR \propto P_th^0.5
cpipe.cluster.X_crp_E = {'X':1.0, 'R_norm':cpipe.cluster.R500} # Assumes large amount of CR (~0.01 expected)

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

--- silent
    True
    <class 'bool'>
--- output_dir
    /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example
    <class 'str'>
--- cosmo
    FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)
    <class 'astropy.cosmology.flrw.FlatLambdaCDM'>
--- name
    Perseus
    <class 'str'>
--- coord
    <SkyCoord (ICRS): (ra, dec) in deg
    (49.94666667, 41.51305556)>
    <class 'astropy.coordinates.sky_coordinate.SkyCoord'>
--- redshift
    0.017284
    <class 'float'>
--- D_ang
    74.8907830317466 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- D_lum
    77.50198024167615 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    620000000000000.0 solMass
    <class 'astropy.units.quantity.Quantity'>
--- R500
    1317.548394363056 kpc
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    1.0080007077672122 deg
    <class 'astropy.units.quantity.Quantity'>
--- R_truncation
    1.9606361149510187

### Define the point source object

In [4]:
# source 1
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}

# Add the source to the model (including the redshift is necesary to account for the EBL)
cpipe.compact_source.add_source(name, spatial, spectral, redshift=cpipe.cluster.redshift)

In [5]:
# source 2
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

# Add the source to the model (including the redshift is necesary to account for the EBL)
cpipe.compact_source.add_source(name, spatial, spectral, redshift=cpipe.cluster.redshift)

In [6]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

--- NGC1275 at z=0.017284
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.95066667 deg>, 'free': False}
         DEC: {'value': <Latitude 41.51166667 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 2.1e-11 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -3.6, 'free': True}
         PivotEnergy: {'value': <Quantity 0.2 TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': False}
--- IC310 at z=0.017284
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.17908333 deg>, 'free': False}
         DEC: {'value': <Latitude 41.325 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -1.95, 'free': True}
         PivotEnergy: {'value': <Quantity 1. TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': Fals

### Define the observations

In [7]:
# One pointing offset +0 +1
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T01:00:00.0', deadc=0.95)

# One pointing offset +sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T01:00:00.0', deadc=0.95)

# One pointing offset -sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T01:00:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 001
        RA-Dec:    49.94666666666666, 42.51305555555555 deg
        GLON-GLAT: 150.0078784158764, -12.426877669678154 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg2, ObsID 002
        RA-Dec:    50.8126920704511, 41.01305555555555 deg
        GLON-GLAT: 151.41415408419417, -13.314435526241189 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-02T00:00:00.0
        tmax: 2020-01-02T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg3, ObsID 003
        RA-Dec:    49.08064126288222, 41.01305

# Run the simulation

In [9]:
cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True,
                        ShowObsDef=True,
                        ShowSkyModel=True,
                        bkgsubtract='NONE',
                        smoothing_FWHM=0.2*u.deg)

----- ObsID to be observed: ['001', '002', '003']
------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Model_Unstack.xml
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/TmpEvents
 startindex ................: 1
 seed ......................: 922479
 ra ........................: 83.63
 dec .......................: 22.51
 rad .......................: 5.0
 tmin ......................: 2020-01-01T00:00:00
 tmax ......................: 2020-01-01T00:30:00
 mjdref ....................: 51544.5
 emin ....

/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:207: RuntimeWarning: invalid value encountered in reciprocal
  F2 = (x * (1 + 0.3/x**beta_e))**-1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/py

!!! WARNING: XSPEC_table.txt not generated, skip Xray flux and Sx


/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.8/site-packages/minot/ClusterTools/cluster_hadronic_emission_kelner2006.py:172: RuntimeWarning: invalid value encountered in reciprocal
  beta = (1.79 + 0.11 * L + 0.008 * L ** 2) ** -1


!!! WARNING: XSPEC_table.txt not generated, skip Xray spectrum
!!! WARNING: XSPEC_table.txt not generated, skip PLOT_MAP_X
CTA_PSF:  0.08162769933172241 , PSF_tot: 0.21601639127202843
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NON

...

----- Quicklook 1/3 done
CTA_PSF:  0.08162769933172241 , PSF_tot: 0.21601639127202843
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events002.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap002.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius ...

Set MJD-END to 58850.000000 from DATE-END'. [astropy.wcs.wcs]


----- Quicklook 2/3 done
CTA_PSF:  0.08162769933172241 , PSF_tot: 0.21601639127202843
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Events003.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Sim_Skymap003.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius ...

Set MJD-END to 58851.000000 from DATE-END'. [astropy.wcs.wcs]


----- Quicklook 3/3 done


# Run the analysis

In [10]:
#----- Method
cpipe.method_binned = True   # Do a binned analysis
cpipe.method_stack  = True   # Stack the event from different observations in a single analysis?
cpipe.method_ana    = '3D'   # 3D or ONOFF analysis

#----- Energy range/binning
cpipe.spec_enumbins = 10
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 10*u.TeV

#----- Imaging
# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = False
# Define the map used for the binned analysis 
cpipe.map_reso          = 0.05*u.deg # Can be increaded if the code is too slow 
cpipe.map_fov           = 3*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)

#----- Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [10]:
#----- Run the analysis
#cpipe.run_analysis(obsID=None,                # A subset of obsid can be analysed
#                   do_like=True,              # Perform likelihood fit?
#                   do_upperlimit=False,       # Compute upper limit?
#                   do_img=True,               # Perform imaging analysis?
#                   do_spec=True,              # Perform Spectral analysis?
#                   do_timing=False,           # Perform lightcurve analysis?
#                   do_expected_output=True,   # Compute the simulated expected output for comparison with recovered?
#                   do_plot=True)              # Plot the results?

In [11]:
cpipe.run_ana_dataprep(overwrite_data=True, overwrite_irfs=True)


            Starting the data preparation             

----- ObsID to be analysed: ['001', '002', '003']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 2.12132034355964
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 10
 phase .....................: NONE
 expr ......................: 
 usethres ..................: USER
 publish ...................: no
 chatter .......

(<gammalib.model.GModels; proxy of <Swig Object of type 'GModels *' at 0x12d24ae70> >,
 <ctools.tools.ctbin; proxy of <Swig Object of type 'ctbin *' at 0x125723ed0> >,
 <ctools.tools.ctbin; proxy of <Swig Object of type 'ctbin *' at 0x124680090> >,
 <ctools.tools.ctexpcube; proxy of <Swig Object of type 'ctexpcube *' at 0x1304a1870> >,
 <ctools.tools.ctpsfcube; proxy of <Swig Object of type 'ctpsfcube *' at 0x108820060> >,
 <ctools.tools.ctbkgcube; proxy of <Swig Object of type 'ctbkgcube *' at 0x1246801e0> >)

In [12]:
cpipe.run_ana_likelihood(refit=False, like_accuracy=0.05, max_iter=50, fix_spat_for_ts=False)


              Starting the likelihood fit             

=== GApplication ===
 Name ......................: ctlike
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Input_Stack.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outmodel ..................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 outcovmat .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_M

(<ctools.tools.ctlike; proxy of <Swig Object of type 'ctlike *' at 0x13088ded0> >,
 <ctools.tools.ctmodel; proxy of <Swig Object of type 'ctmodel *' at 0x12cf041b0> >,
 <ctools.tools.ctmodel; proxy of <Swig Object of type 'ctmodel *' at 0x1088200c0> >)

In [13]:
cpipe.run_ana_imaging(bkgsubtract='NONE', do_TS=False, do_Res=True, do_Skymap=True, do_SourceDet=False)


            Starting the imaging analysis             

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_EventsSelected.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_SkymapTot.fits
 emin ......................: 0.05
 emax ......................: 10
 usepnt ....................: no
 nxpix .....................: 61
 nypix .....................: 61
 binsz .....................: 0.05
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius .................: 1.209600

=== GApplication ===
 Name ......................: csresmap
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output_Cluster.xml
 modcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Cube_Cluster.fits
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_ResmapCluster_SUBDIVSQRT.fits
 ebinalg .......

In [14]:
cpipe.run_ana_spectral(do_Spec=True, do_Butterfly=True, do_Res=True)


            Starting the spectral analysis            

--- Computing spectrum: Perseus
=== GApplication ===
 Name ......................: csspec
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Model_Output.xml
 srcname ...................: Perseus
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Perseus.fits
 method ..........

ZeroDivisionError: float division by zero

In [15]:
cpipe.run_ana_expected_output(profile_reso=0.05*u.deg)


 Starting the calculation of the expected output model

=== GApplication ===
 Name ......................: ctmodel
 Version ...................: 1.7.4
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 incube ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expected_Cluster.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Bkgcube.fits
 caldb .....................: NONE
 irf .......................: NONE
 edisp .....................: no
 outcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Expected_

In [17]:
#cpipe.run_ana_timing()

In [16]:
cpipe.run_ana_plot(profile_log=True, smoothing_FWHM=0.1*u.deg)


 Starting the automatic plots

----- ObsID to be looked at: ['001', '002', '003']

2022-01-21T11:39:12: +============+
2022-01-21T11:39:12: | Parameters |
2022-01-21T11:39:12: +============+
2022-01-21T11:39:12:  inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_ObsDef.xml
2022-01-21T11:39:12:  outds9file ................: [not queried]
2022-01-21T11:39:12:  offset ....................: yes
2022-01-21T11:39:12:  ra ........................: 49.9466666666667
2022-01-21T11:39:12:  dec .......................: 41.5130555555556
2022-01-21T11:39:12:  chatter ...................: 2
2022-01-21T11:39:12:  clobber ...................: yes
2022-01-21T11:39:12:  debug .....................: yes
2022-01-21T11:39:12:  mode ......................: ql
2022-01-21T11:39:12:  logfile ...................: csobsinfo.log
2022-01-21T11:39:12: 
2022-01-21T11:39:12: +==============+
2022-01-21T11:39:12: | Observations |
2022-01-21T11:39:12: +==============+
2022-01-21T11:3

Set MJD-END to 58851.000000 from DATE-END'. [astropy.wcs.wcs]


         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_NGC1275.fits does not exist, no Spectrum_NGC1275 plot
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_IC310.fits does not exist, no Spectrum_IC310 plot
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Spectrum_Residual.fits does not exist, no Spectrum_Residual plot
/Users/adam/Project/CTA/Phys/Outputs/KESACCO_example/Ana_Lightcurve_Perseus.fits does not exist, no Lightcurve_Perseus plot
/Users/adam/Project/CTA/Phys/O

In [17]:
cpipe.run_ana_mcmc_spectrum(reset_mcmc=True, run_mcmc=True, GaussLike=False)


         Starting the MCMC spectrum analysis          

    No existing sampler found
--- MCMC spectrum parameters: 
    Ndim                = 2
    Nwalkers            = 10
    Nsteps              = 1000
    burnin              = 100
    conf                = 68.0
    reset mcmc          = True
    Gaussian likelihood = False
    No pre-existing sampler, start from scratch
--- Runing 1000 MCMC steps


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:58<00:00,  1.86it/s]


param 0 (X_{CRp}): 
   median   = 1.1614922407177262 -0.15063595714788103 +0.19584833115762001
   best-fit = 1.0808845949953874 -0.07002831142554222 +0.2764559768799588
   X_{CRp} = 1.1614922407177262_{-0.15063595714788103}^{0.19584833115762001}
param 1 (\alpha_{CRp}): 
   median   = 2.2237083849303874 -0.028346360592502506 +0.03010543527331544
   best-fit = 2.209418856097226 -0.014056831759341026 +0.04439496410647692
   \alpha_{CRp} = 2.2237083849303874_{-0.028346360592502506}^{0.03010543527331544}


UnboundLocalError: local variable 'smooth' referenced before assignment

In [ ]:
#cpipe.run_ana_mcmc_profile(reset_modelgrid=True,reset_mcmc=True,run_mcmc=True,GaussLike=False,
#                           profile_reso=0.05*u.deg,spatial_range=[0.0,2.0],spatial_npt=11)

In [ ]:
#cpipe.run_ana_mcmc_spectralimaging(reset_modelgrid=True, reset_mcmc=True, run_mcmc=True, GaussLike=False,
#                                   spatial_range=[0.0,2.0], spatial_npt=11, spectral_range=[2.0,3.0], spectral_npt=11,
#                                   bkg_marginalize=False, bkg_spectral_npt=13, bkg_spectral_range=[-0.05,0.05],ps_spectral_npt=19, ps_spectral_range=[-1.0,1.0],
#                                   rm_tmp=False,FWHM=0.1*u.deg,theta=1.0*u.deg,coord=None,profile_reso=0.05*u.deg,includeIC=False)